Let's see if we can make  a model to predict the buybox winner

In [2]:
from mlrepricer import helper
from mlrepricer.oldsql import schemas
from mlrepricer.oldsql.database import SQLite
import pandas as pd
import numpy as np

In [3]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

In [4]:
filter1 = helper.cleanup(df)
norm = helper.normalize(filter1)

In [5]:
len(norm)

238785